# Taxa de rendimento no ensino básico brasileiro
Este notebook será usado para fazer análises baseadas no dataframe disponibilizado pelo governo federal sobre o rendimentos dos alunos de diversas escolas espalhadas pelo país e que pode ser verificado através do link http://dados.gov.br/dataset/taxas-de-rendimento-escolar-na-educacao-basica?fbclid=IwAR1k-zIfTDd7K6rWWhIYY9JPyfkjYI8Pm83tY7_K29ZWntVCx5vMGoPCpMM 

O dataset usado é "Taxas - município 2010"
Serão apresentados os dados recolhidos, as hipóteses criadas, bem como se foram validadas ou não e algumas visualizações e inferências importantes. 

In [5]:
import pandas as pd

In [6]:
file = r'data/TX RENDIMENTO MUNICIPIOS 2010.xls'
column_labels = [
    'ano', 'regiao', 'uf', 'codigo_municipio', 'nome_municipio', 'localizacao', 'rede',
    'aprovacao_fundamental_1', 'aprovacao_fundamental_2', 'aprovacao_fundamental_3',
    'aprovacao_fundamental_4', 'aprovacao_fundamental_5', 'aprovacao_fundamental_6',
    'aprovacao_fundamental_7', 'aprovacao_fundamental_8', 'aprovacao_fundamental_9',
    'aprovacao_fundamental_1:5', 'aprovacao_fundamental_6:9', 'aprovacao_fundamental_1:9',
    'aprovacao_medio_1', 'aprovacao_medio_2', 'aprovacao_medio_3', 'aprovacao_medio_4',
    'aprovacao_medio_0', 'aprovacao_medio_0:4',
    'reprovacao_fundamental_1', 'reprovacao_fundamental_2', 'reprovacao_fundamental_3',
    'reprovacao_fundamental_4', 'reprovacao_fundamental_5', 'reprovacao_fundamental_6',
    'reprovacao_fundamental_7', 'reprovacao_fundamental_8', 'reprovacao_fundamental_9',
    'reprovacao_fundamental_1:5', 'reprovacao_fundamental_6:9', 'reprovacao_fundamental_1:9',
    'reprovacao_medio_1', 'reprovacao_medio_2', 'reprovacao_medio_3', 'reprovacao_medio_4',
    'reprovacao_medio_0', 'reprovacao_medio_0:4',
    'abandono_fundamental_1', 'abandono_fundamental_2', 'abandono_fundamental_3',
    'abandono_fundamental_4', 'abandono_fundamental_5', 'abandono_fundamental_6',
    'abandono_fundamental_7', 'abandono_fundamental_8', 'abandono_fundamental_9',
    'abandono_fundamental_1:5', 'abandono_fundamental_6:9', 'abandono_fundamental_1:9',
    'abandono_medio_1', 'abandono_medio_2', 'abandono_medio_3', 'abandono_medio_4',
    'abandono_medio_0', 'abandono_medio_0:4',
]
df_municipios = pd.read_excel(file, skiprows=8, nrows=65460, names=column_labels, na_values='--')

df_municipios.head()

,ano,regiao,uf,codigo_municipio,nome_municipio,localizacao,rede,aprovacao_fundamental_1,aprovacao_fundamental_2,aprovacao_fundamental_3,...,abandono_fundamental_9,abandono_fundamental_1:5,abandono_fundamental_6:9,abandono_fundamental_1:9,abandono_medio_1,abandono_medio_2,abandono_medio_3,abandono_medio_4,abandono_medio_0,abandono_medio_0:4
0,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Estadual,100.0,35.1,70.6,...,NaN,3.4,6.0,4.2,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Municipal,97.9,88.3,88.6,...,11.0,1.2,10.3,6.1,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Publico,98.1,79.3,86.0,...,11.0,1.6,10.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Total,98.1,79.3,86.0,...,11.0,1.6,10.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Total,Estadual,98.2,83.3,82.0,...,8.2,2.0,9.2,6.1,18.3,12.1,8.4,NaN,NaN,14.1


In [17]:
df_municipios['localizacao'] = df_municipios.localizacao.astype('category')
df_localizacao = df_municipios[[
    'localizacao',
    'aprovacao_fundamental_1', 'aprovacao_fundamental_2', 'aprovacao_fundamental_3',
    'aprovacao_fundamental_4', 'aprovacao_fundamental_5', 'aprovacao_fundamental_6',
    'aprovacao_fundamental_7', 'aprovacao_fundamental_8', 'aprovacao_fundamental_9',
    'aprovacao_fundamental_1:5', 'aprovacao_fundamental_6:9', 'aprovacao_fundamental_1:9',
    'aprovacao_medio_1', 'aprovacao_medio_2', 'aprovacao_medio_3', 'aprovacao_medio_4',
    'aprovacao_medio_0', 'aprovacao_medio_0:4',
    'reprovacao_fundamental_1', 'reprovacao_fundamental_2', 'reprovacao_fundamental_3',
    'reprovacao_fundamental_4', 'reprovacao_fundamental_5', 'reprovacao_fundamental_6',
    'reprovacao_fundamental_7', 'reprovacao_fundamental_8', 'reprovacao_fundamental_9',
    'reprovacao_fundamental_1:5', 'reprovacao_fundamental_6:9', 'reprovacao_fundamental_1:9',
    'reprovacao_medio_1', 'reprovacao_medio_2', 'reprovacao_medio_3', 'reprovacao_medio_4',
    'reprovacao_medio_0', 'reprovacao_medio_0:4',
    'abandono_fundamental_1', 'abandono_fundamental_2', 'abandono_fundamental_3',
    'abandono_fundamental_4', 'abandono_fundamental_5', 'abandono_fundamental_6',
    'abandono_fundamental_7', 'abandono_fundamental_8', 'abandono_fundamental_9',
    'abandono_fundamental_1:5', 'abandono_fundamental_6:9', 'abandono_fundamental_1:9',
    'abandono_medio_1', 'abandono_medio_2', 'abandono_medio_3', 'abandono_medio_4',
    'abandono_medio_0', 'abandono_medio_0:4',
]]
df_localizacao = df_localizacao.groupby('localizacao'). mean()
df_localizacao.head()

,aprovacao_fundamental_1,aprovacao_fundamental_2,aprovacao_fundamental_3,aprovacao_fundamental_4,aprovacao_fundamental_5,aprovacao_fundamental_6,aprovacao_fundamental_7,aprovacao_fundamental_8,aprovacao_fundamental_9,aprovacao_fundamental_1:5,...,abandono_fundamental_9,abandono_fundamental_1:5,abandono_fundamental_6:9,abandono_fundamental_1:9,abandono_medio_1,abandono_medio_2,abandono_medio_3,abandono_medio_4,abandono_medio_0,abandono_medio_0:4
localizacao,,,,,,,,,,,,,,,,,,,,,
Rural,94.846063,86.915056,85.554859,88.415854,88.561550,79.073508,83.394576,85.985918,89.519375,88.332337,...,5.236150,2.130268,5.676474,3.004601,12.228783,9.247710,6.027150,6.725510,12.073684,9.719079
Total,96.242314,89.577990,88.257113,90.602716,90.545561,80.735100,83.063811,85.195183,87.078913,90.646913,...,4.941224,1.568586,5.003313,3.101040,12.269891,9.283736,6.516254,5.710033,13.901822,9.838139
Urbana,96.605978,90.042026,88.434791,90.746788,90.597839,80.641993,82.699107,84.856685,86.527746,90.899661,...,4.969642,1.509545,4.967374,3.217923,12.329423,9.353851,6.622767,5.767959,14.344348,9.916634


In [21]:
df_localizacao_rede = df_municipios[[
    'localizacao', 'rede',
    'aprovacao_fundamental_1', 'aprovacao_fundamental_2', 'aprovacao_fundamental_3',
    'aprovacao_fundamental_4', 'aprovacao_fundamental_5', 'aprovacao_fundamental_6',
    'aprovacao_fundamental_7', 'aprovacao_fundamental_8', 'aprovacao_fundamental_9',
    'aprovacao_fundamental_1:5', 'aprovacao_fundamental_6:9', 'aprovacao_fundamental_1:9',
    'aprovacao_medio_1', 'aprovacao_medio_2', 'aprovacao_medio_3', 'aprovacao_medio_4',
    'aprovacao_medio_0', 'aprovacao_medio_0:4',
    'reprovacao_fundamental_1', 'reprovacao_fundamental_2', 'reprovacao_fundamental_3',
    'reprovacao_fundamental_4', 'reprovacao_fundamental_5', 'reprovacao_fundamental_6',
    'reprovacao_fundamental_7', 'reprovacao_fundamental_8', 'reprovacao_fundamental_9',
    'reprovacao_fundamental_1:5', 'reprovacao_fundamental_6:9', 'reprovacao_fundamental_1:9',
    'reprovacao_medio_1', 'reprovacao_medio_2', 'reprovacao_medio_3', 'reprovacao_medio_4',
    'reprovacao_medio_0', 'reprovacao_medio_0:4',
    'abandono_fundamental_1', 'abandono_fundamental_2', 'abandono_fundamental_3',
    'abandono_fundamental_4', 'abandono_fundamental_5', 'abandono_fundamental_6',
    'abandono_fundamental_7', 'abandono_fundamental_8', 'abandono_fundamental_9',
    'abandono_fundamental_1:5', 'abandono_fundamental_6:9', 'abandono_fundamental_1:9',
    'abandono_medio_1', 'abandono_medio_2', 'abandono_medio_3', 'abandono_medio_4',
    'abandono_medio_0', 'abandono_medio_0:4',
]]
df_localizacao_rede = df_localizacao_rede.groupby(['localizacao', 'rede']).mean()
df_localizacao_rede

aprovacao_fundamental_1  aprovacao_fundamental_2  \
localizacao rede                                                           
Rural       Estadual                  95.000967                87.969600   
            Federal                   92.300000               100.000000   
            Municipal                 94.814978                86.704610   
            Particular                95.283721                92.755797   
            Publico                   94.842137                86.776551   
            Total                     94.832293                86.812069   
Total       Estadual                  96.568655                90.477119   
            Federal                   98.300000                97.007143   
            Municipal                 95.848166                87.772636   
            Particular                97.873166                97.549834   
            Publico                   95.956472                88.312489   
            Total                     96.068992                88.705180   
Urbana      Estadual                  97.196012                91.014066   
            Federal                   98.728571                96.776923   
            Municipal                 96.113256                87.948850   
            Particular                97.946546                97.628858   
            Publico                   96.347249                88.867800   
            Total                     96.511336                89.438087   

                        aprovacao_fundamental_3  aprovacao_fundamental_4  \
localizacao rede                                                           
Rural       Estadual                  88.793916                89.657568   
            Federal                   90.900000               100.000000   
            Municipal                 85.058237                88.149515   
            Particular                92.027536                93.592029   
            Publico                   85.269041                88.290234   
            Total                     85.287788                88.318423   
Total       Estadual                  89.600102                90.937984   
            Federal                   96.692308                94.833333   
            Municipal                 86.084148                89.055378   
            Particular                97.950230                98.138865   
            Publico                   86.643217                89.494339   
            Total                     87.124996                89.859337   
Urbana      Estadual                  89.892167                91.216228   
            Federal                   97.175000                94.363636   
            Municipal                 85.803699                88.786992   
            Particular                98.060639                98.212489   
            Publico                   86.898540                89.702069   
            Total                     87.582462                90.229413   

                        aprovacao_fundamental_5  aprovacao_fundamental_6  \
localizacao rede                                                           
Rural       Estadual                  88.809991                83.365751   
            Federal                   70.600000                90.000000   
            Municipal                 88.442581                77.030984   
            Particular                91.942308                89.983495   
            Publico                   88.531264                78.844223   
            Total                     88.548986                78.942172   
Total       Estadual                  89.938914                81.362484   
            Federal                   91.721429                87.481818   
            Municipal                 89.216947                75.609413   
            Particular                97.978857                95.699753   
            Publico                   89.580386                79.3465

In [22]:
df_municipios['regiao'] = df_municipios.regiao.astype('category')
df_regiao = df_municipios[[
    'regiao',
    'aprovacao_fundamental_1', 'aprovacao_fundamental_2', 'aprovacao_fundamental_3',
    'aprovacao_fundamental_4', 'aprovacao_fundamental_5', 'aprovacao_fundamental_6',
    'aprovacao_fundamental_7', 'aprovacao_fundamental_8', 'aprovacao_fundamental_9',
    'aprovacao_fundamental_1:5', 'aprovacao_fundamental_6:9', 'aprovacao_fundamental_1:9',
    'aprovacao_medio_1', 'aprovacao_medio_2', 'aprovacao_medio_3', 'aprovacao_medio_4',
    'aprovacao_medio_0', 'aprovacao_medio_0:4',
    'reprovacao_fundamental_1', 'reprovacao_fundamental_2', 'reprovacao_fundamental_3',
    'reprovacao_fundamental_4', 'reprovacao_fundamental_5', 'reprovacao_fundamental_6',
    'reprovacao_fundamental_7', 'reprovacao_fundamental_8', 'reprovacao_fundamental_9',
    'reprovacao_fundamental_1:5', 'reprovacao_fundamental_6:9', 'reprovacao_fundamental_1:9',
    'reprovacao_medio_1', 'reprovacao_medio_2', 'reprovacao_medio_3', 'reprovacao_medio_4',
    'reprovacao_medio_0', 'reprovacao_medio_0:4',
    'abandono_fundamental_1', 'abandono_fundamental_2', 'abandono_fundamental_3',
    'abandono_fundamental_4', 'abandono_fundamental_5', 'abandono_fundamental_6',
    'abandono_fundamental_7', 'abandono_fundamental_8', 'abandono_fundamental_9',
    'abandono_fundamental_1:5', 'abandono_fundamental_6:9', 'abandono_fundamental_1:9',
    'abandono_medio_1', 'abandono_medio_2', 'abandono_medio_3', 'abandono_medio_4',
    'abandono_medio_0', 'abandono_medio_0:4',
]]
df_regiao = df_regiao.groupby('regiao'). mean()
df_regiao.head()

,aprovacao_fundamental_1,aprovacao_fundamental_2,aprovacao_fundamental_3,aprovacao_fundamental_4,aprovacao_fundamental_5,aprovacao_fundamental_6,aprovacao_fundamental_7,aprovacao_fundamental_8,aprovacao_fundamental_9,aprovacao_fundamental_1:5,...,abandono_fundamental_9,abandono_fundamental_1:5,abandono_fundamental_6:9,abandono_fundamental_1:9,abandono_medio_1,abandono_medio_2,abandono_medio_3,abandono_medio_4,abandono_medio_0,abandono_medio_0:4
regiao,,,,,,,,,,,,,,,,,,,,,
Centro_Oeste,97.361488,88.238283,90.285062,92.532572,92.955058,85.511917,87.179449,88.013854,88.136040,92.010557,...,5.127678,1.099678,4.206177,2.579946,11.769612,9.419091,7.696388,5.927273,10.172010,10.010635
Nordeste,93.529492,85.761026,82.512602,83.702894,84.486823,71.464394,77.639395,81.788061,85.086655,85.163212,...,7.490222,3.180603,8.183833,5.298652,17.479402,13.271425,9.172882,6.121935,17.083205,13.851667
Norte,93.985616,82.336296,82.430501,85.017653,86.815807,79.172616,82.556569,85.053001,86.628936,85.547181,...,6.562625,3.414434,6.843735,4.688876,14.673701,11.608712,8.307824,7.331579,10.834356,12.136957
Sudeste,97.232025,94.238473,91.265425,95.638039,94.271298,85.985374,87.713576,88.358478,88.434983,94.217150,...,3.129528,0.592310,3.115220,1.658757,8.305524,6.448239,4.409163,6.764308,14.564315,6.745536
Sul,98.699818,90.565357,92.188415,94.394732,94.269273,86.254000,84.193763,85.771044,89.623988,93.971614,...,2.658184,0.262483,2.235907,1.242740,9.417891,6.453334,4.331583,4.728601,12.214414,7.229721
